### Objective: To test analysis with nc file and write a pipeline function to do all the analysis

Istvan recently installed Amber in a docker and ran a simulation of an antibody using the docker. Implicit Solvent. 

In [1]:
import nglview as nv
import pandas as pd
from pathlib import Path
import mdtraj as md
import pytraj as pt

from vir_md_analysis.utils import ls

##### Load the NC file. 

In [2]:
data_dir = Path().cwd().parent.parent / 'vir_md_analysis' / 'data'
ls(data_dir)

['8md_prod.info',
 '8md_prod.rst',
 '8md_prod.out',
 '8md_prod.nc',
 'MAbFv.parm7',
 'alpha_RBD_B38.psf',
 'README.md',
 'MAbFv.parm7.hmass.parm7',
 '4md.nc',
 'alpha_RBD_B38_500ns.dcd',
 'prep4.pdb',
 'look_and_say.dat',
 '2md.nc',
 '3md.nc']

In [3]:
def load_mdtraj(trajectory_file: str | Path, topology_file: str | Path) -> md.Trajectory:
    """Load a trajectory using mdtraj.
    trajectory_file: str or Path
        Path to the trajectory file (e.g., .dcd, .nc).
    topology_file: str or Path
        Path to the topology file (e.g., .pdb, .prmtop).
    Returns:
        md.Trajectory object.
    """

    if isinstance(trajectory_file, str):
        trajectory_file = Path(trajectory_file)
    if isinstance(topology_file, str):
        topology_file = Path(topology_file)
    # Check if the files exist

    if not trajectory_file.exists():
        raise FileNotFoundError(f"Trajectory file {trajectory_file} does not exist.")
    if not topology_file.exists():
        raise FileNotFoundError(f"Topology file {topology_file} does not exist.")

    if trajectory_file.suffix == '.nc':
        traj = md.load_netcdf(trajectory_file, top=topology_file)
    elif trajectory_file.suffix == '.dcd':
        traj = md.load_dcd(trajectory_file, top=topology_file)
    else:
        raise ValueError("Unsupported trajectory file format. Only .nc and .dcd are supported.")
    return traj


def load_pytraj(trajectory_file: str | Path, topology_file: str | Path) -> pt.Trajectory:
    """Load a trajectory using pytraj.
    trajectory_file: str or Path
        Path to the trajectory file (e.g., .dcd, .nc).
    topology_file: str or Path
        Path to the topology file (e.g., .pdb, .prmtop).
    Returns:
        pt.Trajectory object.
    """
    if isinstance(trajectory_file, str):
        trajectory_file = Path(trajectory_file)
    if isinstance(topology_file, str):
        topology_file = Path(topology_file)

    # Check if the files exist
    if not trajectory_file.exists():
        raise FileNotFoundError(f"Trajectory file {trajectory_file} does not exist.")
    if not topology_file.exists():
        raise FileNotFoundError(f"Topology file {topology_file} does not exist.")

    traj = pt.load(str(trajectory_file), top=str(topology_file))
    return traj

In [4]:
nc = data_dir/ '8md_prod.nc'
top = data_dir/'MAbFv.parm7.hmass.parm7'
traj = load_mdtraj(nc, top)

In [5]:
pt_traj = load_pytraj(nc, top)

##### Write the function to get all the data of interest and return a DataFrame

This function will load the NC file and topology file into mdtraj and then pytraj. For each library, it will extract the appropriate features and return a DataFrame. m

In [6]:
# A dictionary that holds reference to the features of interest. 
feature_func_map = {'mdtraj': [], 'pytraj': []}

In [7]:
from collections import defaultdict

def extract_features(trajectory_file: Path | str,
                     topology_file: Path | str,
                     library: str='mdtraj',
                     features: list[callable] | None = None,
                     system_name: str | None = None):
    """Extract features from a trajectory using either mdtraj or pytraj.
    Arguments:
        trajectory_file: Path or str
            Path to the trajectory file (e.g., .dcd, .nc).
        topology_file: Path or str
            Path to the topology file (e.g., .pdb, .prmtop).
        library: str
            The library to use for loading the trajectory. Options are 'mdtraj' or 'pytraj'.
        features: list of callables or None
            List of feature extraction functions to apply. If None, default features for the library will be used.
        system_name: str or None
            Name of the system for the output DataFrame. If None, the name will be derived from the trajectory file name.
    Returns:
        pd.DataFrame
            DataFrame containing the extracted features with the system name as the first column.
    
    """
    if library == 'mdtraj':
        traj = load_mdtraj(trajectory_file, topology_file)
    elif library == 'pytraj':
        traj = load_pytraj(trajectory_file, topology_file)
    else:
        raise ValueError("Unsupported library. Use 'mdtraj' or 'pytraj'.")

    if features is None:
        features = feature_func_map[library]

    feature_values = {}
    # consider using dask or multiprocessing for large trajectories
    for func in features: 
        results = func(traj)
        feature_values[func.__name__] = results 

    feature_df = pd.DataFrame(feature_values)
    if not system_name:
        system_name = trajectory_file.stem
    
    feature_df.insert(0, 'system', system_name)
    return feature_df
